<a href="https://colab.research.google.com/github/VladislavEpifanow/BoysNextDoor/blob/main/Predictive_analytics%2C_Methods_research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm

import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error

import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs
from scipy.optimize import minimize

import matplotlib.pyplot as plt

In [ ]:
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive') 

In [ ]:
import urllib.request, json 
with urllib.request.urlopen("https://teplo.set-energo.ru/elev/graph_data_json/5/%D0%A2%D0%B5%D0%BC%D0%BF%D0%B5%D1%80%D0%B0%D1%82%D1%83%D1%80%D0%B0%20%D0%93%D0%92C/2020-01-01/2022-08-05%2000:00") as url:
    data = json.load(url)
    print(data)

In [ ]:
from datetime import datetime, timedelta

In [ ]:
time_series = data['SeriesData'][0]

In [ ]:
print("Всего {} измерений, из которых {} - пустых ".format(len(time_series),time_series.count(None)))
print("Процент пустых измерений составляет : {} %".format(time_series.count(None)/len(time_series)*100))

In [ ]:
print("Start date : {}, End date : {}".format(data['NavStart'],data['NavEnd']))

In [ ]:
start_date = datetime(*[int(x) for x in data['NavStart'].split('-')])

In [ ]:
end_date = datetime(*[int(x) for x in data['NavEnd'].split('-')])

In [ ]:
def get_timescales(start_date,end_date,ts_size):
  time_delta = (end_date - start_date)/ts_size
  return [start_date + time_delta*i for i in range(ts_size)]

In [ ]:
datas = get_timescales(start_date,end_date,len(time_series))

In [ ]:
from matplotlib.dates import DateFormatter

In [ ]:
data['GraphName']

In [ ]:
time_series

In [ ]:
# Построение графика временного ряда 
plt.style.use('seaborn')
plt.figure(figsize=(25,10))
date_format = DateFormatter('%d-%m-%Y')
plt.gca().xaxis.set_major_formatter(date_format)
plt.plot_date(datas, time_series,linestyle ='solid')
plt.xticks(fontsize=17)
plt.yticks(fontsize=17)
plt.title(data['GraphName'], fontdict = {'fontsize' : 30})
plt.tight_layout()
plt.show()

Как видно - 18 процентов данных являются пустыми - что не очень хорошо. Нужно решать, как избавиться от этого


In [ ]:
time_series = np.array(data['SeriesData'][0])

In [ ]:
df = pd.Series(time_series)

In [ ]:
# TODO: Не всегда интерполируется с первого раза
df = df.interpolate()

In [ ]:
time_series = np.copy(df.to_numpy(dtype=float))

In [ ]:
len(np.argwhere(np.isnan(time_series)))

In [ ]:
# Построение графика временного ряда c интерполяцией
plt.style.use('seaborn')
plt.figure(figsize=(25,10))
date_format = DateFormatter('%d-%m-%Y')
plt.gca().xaxis.set_major_formatter(date_format)
plt.plot_date(datas, time_series,linestyle ='solid')
plt.xticks(fontsize=17)
plt.yticks(fontsize=17)
plt.title(data['GraphName'], fontdict = {'fontsize' : 30})
plt.tight_layout()
plt.show()

In [ ]:
len(time_series)

In [ ]:
def prepareTimeSeriesDataset(time_series,features):
  X = []
  y = []
  for i in range(0,len(time_series) - (features + 1)):
    if time_series[i:i + features - 1 ].all() and time_series[i+features] is not None:
      X.append(time_series[i:i + features - 1 ])
      y.append(time_series[i+features])
  return np.array(X), np.array(y)

In [ ]:
X,y = prepareTimeSeriesDataset(time_series,15)

In [ ]:
from sklearn.linear_model import ElasticNet,LinearRegression

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2)

In [ ]:
regr = ElasticNet()
regr.fit(X_train,y_train)
print("Средняя квадратичная ошибка составляет: {}".format(mean_squared_error(y_test,regr.predict(X_test))))

In [ ]:
for i in range(X_train.shape[0]):
  for j in range(X_train.shape[1]):
    if X_train[i,j] is None:
      print(i,j)
      

In [ ]:
time_series[-15:]

In [ ]:
def make_time_series_prediction(model,future_points,time_series,n_features,start_time,time_interval):
  predicted_time_series = []
  predicted_dates = []
  for i in time_series[-n_features:]:
    predicted_time_series.append(i)
  for j in range(future_points):
    predicted_time_series.append(model.predict(np.array(predicted_time_series[j:j+n_features]).reshape(1, -1))[0])
  for it in range(len(predicted_time_series)):
    predicted_dates.append(start_time + it*time_interval)
  return predicted_time_series, predicted_dates


In [ ]:
regr.predict(X_test[-1].reshape(1,-1))[0]

In [ ]:
time_series[-6:]

In [ ]:
time_series[-200:-(200-14)]

In [ ]:
start_time = datas[-200]

In [ ]:
start_time

In [ ]:
time_interval = datas[-99] - datas[-100]

In [ ]:
predictions, pred_dates = make_time_series_prediction(regr,400,time_series[-200:-(200-14)],14,start_time,time_interval)

In [ ]:
# Построение графика временного ряда 
plt.style.use('seaborn')
plt.figure(figsize=(25,10))
date_format = DateFormatter('%d-%m-%Y')
plt.gca().xaxis.set_major_formatter(date_format)
plt.plot_date(datas, time_series,linestyle ='solid')
plt.plot_date(pred_dates, predictions,linestyle ='solid')
plt.xticks(fontsize=17)
plt.yticks(fontsize=17)
plt.title(data['GraphName'], fontdict = {'fontsize' : 30})
plt.tight_layout()
plt.show()

In [ ]:
y.mean()

Пока что получается, что прогноз стремиться к какому то одному значение( чет типа среднего)

Предлагаемые следующие действия:
1.   Прогон моделей на искусственных данных без пропусков
2.   Добавление разных типов моделей, а именно: Статистические, нейронные сети
3.   Замена пропусков


In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
num_shape = 2985

train = time_series[:num_shape]
test = time_series[num_shape:]

In [ ]:
sc = MinMaxScaler(feature_range = (0, 1))
train_scaled = sc.fit_transform(train.reshape(-1, 1))

In [ ]:
X_train = []

#Price on next day
y_train = []

window = 60

for i in range(window, num_shape):
    X_train_ = np.reshape(train_scaled[i-window:i, 0], (window, 1))
    X_train.append(X_train_)
    y_train.append(train_scaled[i, 0])
X_train = np.stack(X_train)
y_train = np.stack(y_train)

In [ ]:
X_train

Теперь попробуем подключить нейронную сеть LSTM из библиотеки Keras

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GRU
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from keras.optimizers import adam_v2

In [ ]:
# Инициализация Рекурентной нейронной сети
model = Sequential()

model.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
model.add(Dropout(0.2))

model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))

model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))

model.add(LSTM(units = 50))
model.add(Dropout(0.2))

# Выходной слой
model.add(Dense(units = 1))
model.summary()

In [ ]:
model.compile(optimizer = 'adam', loss = 'mean_squared_error')
model.fit(X_train, y_train, epochs = 500, batch_size = 16);

Теперь попробуем предсказать выход модели LSTM:

In [ ]:
df_volume = np.vstack((train.reshape(-1,1), test.reshape(-1,1)))

inputs = df_volume[df_volume.shape[0] - test.shape[0] - window:]
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)

num_2 = df_volume.shape[0] - num_shape + window

X_test = []

for i in range(window, num_2):
    X_test_ = np.reshape(inputs[i-window:i, 0], (window, 1))
    X_test.append(X_test_)
    
X_test = np.stack(X_test)

In [ ]:
predict = model.predict(X_test)
predict = sc.inverse_transform(predict)

In [ ]:
diff = predict - test

print("MSE:", np.mean(diff**2))
print("MAE:", np.mean(abs(diff)))
print("RMSE:", np.sqrt(np.mean(diff**2)))

In [ ]:
# Построение графика временного ряда 
plt.style.use('seaborn')
plt.figure(figsize=(25,10))
date_format = DateFormatter('%d-%m-%Y')
plt.gca().xaxis.set_major_formatter(date_format)
plt.plot_date(datas[-100:], time_series[-100:],linestyle ='solid',color ='red',label = 'Реальное значение')
plt.plot_date(datas[-19:], predict,linestyle ='solid',color = 'blue', label = 'Предсказанное значение')
plt.xticks(fontsize=17)
plt.yticks(fontsize=17)
plt.title(data['GraphName'], fontdict = {'fontsize' : 30})
plt.xlabel('Дата')
plt.legend()
plt.ylabel(data['GraphYAxisName'])
plt.tight_layout()
plt.show()

In [ ]:
model.save('/content/drive/MyDrive/GachiDashboard')